In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_manufacturer = df['Manufacturer'].tolist()
list_part_number = df['Part_Number'].tolist()
list_info = df['Info'].tolist()

print('总数量：' + str(len(list_info)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for manufacturer, part_number, info in zip(list_manufacturer, list_part_number, list_info):
    work.put_nowait((manufacturer, part_number, info))

def crawler():
    global df
    
    while not work.empty():
        manufacturer, part_number, info = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(info,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                if html.xpath('//span[contains(@class, "listing-final-manufacturer")]/text()')[0] != manufacturer or html.xpath('//span[contains(@id, "vew_partnumber[1]")]/text()')[0] != part_number:
                    continue
                    
                # = = = = = = = = = = = = = = =

                list_tr = html.xpath('//table[contains(@class, "moreinfotable")]/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Info': info}])
                
                for tr in list_tr[1:]:
                    df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip().lower()] = tr.xpath('./td[2]/text()')[0].strip()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Info': info}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(info +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./specification_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：364

https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773489  <->  [ok] - 剩余数量：344 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773497  <->  [ok] - 剩余数量：343 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773465  <->  [ok] - 剩余数量：342 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773481  <->  [ok] - 剩余数量：341 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773545  <->  [ok] - 剩余数量：340 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773493  <->  [ok] - 剩余数量：339 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773485  <->  [ok] - 剩余数量：338 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773537  <->  [ok] - 剩余数量：337 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773501  <->  [ok] - 剩余数量：336 - 当前时间：17:38:35
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773521  <->  [ok] - 剩余数量：335 - 当前时间：17:38:36
h

https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773897  <->  [ok] - 剩余数量：260 - 当前时间：17:39:03
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773977  <->  [ok] - 剩余数量：259 - 当前时间：17:39:04
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773853  <->  [ok] - 剩余数量：258 - 当前时间：17:39:04
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773729  <->  [ok] - 剩余数量：257 - 当前时间：17:39:05
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773917  <->  [ok] - 剩余数量：256 - 当前时间：17:39:05
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773873  <->  [ok] - 剩余数量：255 - 当前时间：17:39:06
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774005  <->  [ok] - 剩余数量：254 - 当前时间：17:39:06
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773945  <->  [ok] - 剩余数量：253 - 当前时间：17:39:07
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14773657  <->  [ok] - 剩余数量：252 - 当前时间：17:39:08
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774017  <->  [ok] - 剩余数量：251 - 当前时间：17:39:08
https://ww

https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774341  <->  [ok] - 剩余数量：177 - 当前时间：17:39:49
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774321  <->  [ok] - 剩余数量：176 - 当前时间：17:39:49
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774269  <->  [ok] - 剩余数量：175 - 当前时间：17:39:49
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774345  <->  [ok] - 剩余数量：174 - 当前时间：17:39:49
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774397  <->  [ok] - 剩余数量：173 - 当前时间：17:39:50
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774409  <->  [ok] - 剩余数量：172 - 当前时间：17:39:50
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774417  <->  [ok] - 剩余数量：171 - 当前时间：17:39:51
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774385  <->  [ok] - 剩余数量：170 - 当前时间：17:39:52
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774309  <->  [ok] - 剩余数量：169 - 当前时间：17:39:52
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774353  <->  [ok] - 剩余数量：168 - 当前时间：17:39:52
https://ww

https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774785  <->  [ok] - 剩余数量：93 - 当前时间：17:40:27
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774761  <->  [ok] - 剩余数量：92 - 当前时间：17:40:27
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774777  <->  [ok] - 剩余数量：91 - 当前时间：17:40:27
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774781  <->  [ok] - 剩余数量：90 - 当前时间：17:40:28
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774677  <->  [ok] - 剩余数量：89 - 当前时间：17:40:28
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774773  <->  [ok] - 剩余数量：88 - 当前时间：17:40:31
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774801  <->  [ok] - 剩余数量：87 - 当前时间：17:40:32
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774821  <->  [ok] - 剩余数量：86 - 当前时间：17:40:32
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774757  <->  [ok] - 剩余数量：85 - 当前时间：17:40:32
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14774725  <->  [ok] - 剩余数量：84 - 当前时间：17:40:33
https://www.rockauto

https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775193  <->  [ok] - 剩余数量：9 - 当前时间：17:41:09
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775141  <->  [ok] - 剩余数量：8 - 当前时间：17:41:09
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775185  <->  [ok] - 剩余数量：7 - 当前时间：17:41:09
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775145  <->  [ok] - 剩余数量：6 - 当前时间：17:41:09
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775181  <->  [ok] - 剩余数量：5 - 当前时间：17:41:09
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775177  <->  [ok] - 剩余数量：4 - 当前时间：17:41:10
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775153  <->  [ok] - 剩余数量：3 - 当前时间：17:41:10
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775197  <->  [ok] - 剩余数量：2 - 当前时间：17:41:11
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775161  <->  [ok] - 剩余数量：1 - 当前时间：17:41:11
https://www.rockauto.com/en/moreinfo.php?pt=5562&pk=14775229  <->  [ok] - 剩余数量：0 - 当前时间：17:41:13
https://www.rockauto.com/en/mo